In [1]:
import torch
from gpn.phylogpn import model, tokenizer  # Wrapper around HuggingFace

# Example data
seqs = ["TATAAA", "GGCCAATCT", "CACGTG", "AGGTCACGT", "GCCAGCC", "GGGGATTTCC"]

# Output length is input length minus 480 (the receptive field size minus 1)
pad_token = tokenizer.pad_token
pad_size = 481 // 2
pad_sequence = lambda seq: pad_token * pad_size + seq + pad_token * pad_size
padded_seqs = [pad_sequence(seq) for seq in seqs]
input_tensor = tokenizer(padded_seqs, return_tensors="pt", padding=True)["input_ids"]

with torch.no_grad():
    padded_embeddings = model.get_embeddings(input_tensor)
    padded_logits = model(
        input_tensor
    )  # These are log rate parameters for the F81 model

embeddings = []
logits = []

for i in range(len(seqs)):
    length = len(seqs[i])
    embeddings.append(padded_embeddings[i, :length])
    logits.append({})

    for k in "ACGT":
        logits[-1][k] = padded_logits[k][i, :length]

/accounts/projects/yss/carlos_albors/gpn/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/accounts/projects/yss/carlos_albors/gpn/venv/lib/python3.12/site-packages/transformers/utils/hub.py:106: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
A new version of the following files was downloaded from https://huggingface.co/songlab/PhyloGPN:
- tokenization_phylogpn.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/songlab/PhyloGPN:
- configuration_phylogpn.py
. Make sure to double-check they do not contain any added malicious code. 

In [2]:
# `embeddings` is a list of tensors, one per item in the batch, each containing embeddings for each position in the sequence
embeddings[0]

tensor([[ 0.2399,  0.2717,  0.1193,  ..., -0.2694,  1.2095, -0.8143],
        [-0.3725,  1.2785, -0.0264,  ..., -0.5638, -0.1288, -1.1086],
        [-0.7089,  0.8101, -1.5966,  ..., -0.4742,  1.5401, -0.8086],
        [-0.6684,  1.1999, -1.6881,  ...,  0.0269, -0.1773,  0.2568],
        [-0.1105,  0.2217, -0.3428,  ..., -0.4531,  1.1346, -0.8851],
        [-0.8363,  1.2306,  0.3084,  ..., -0.7149,  0.5221,  0.4968]])

In [3]:
# `logits` is a list of dictionaries, one per item in the batch, each containing the log rate parameters for the F81 model
logits[0]

{'A': tensor([ 0.3503,  0.1109,  0.5503, -0.1649,  0.6182,  0.1694]),
 'C': tensor([-0.0261,  0.3540, -0.3431,  0.5716,  0.4910,  0.3965]),
 'G': tensor([0.4168, 0.4671, 0.3895, 0.1255, 0.1161, 0.3556]),
 'T': tensor([-0.2667,  0.0614, -0.2464,  0.5956, -0.1491, -0.1738])}

In [ ]:
# Get likelihoods

likelihood_list = []

for logit_dict in logits:
    logit_tensor = torch.stack([logit_dict[k] for k in "ACGT"], dim=1)
    likelihood_tensor = torch.softmax(logit_tensor, dim=1)
    likelihood_dict = {k: likelihood_tensor[i] for i, k in enumerate("ACGT")}
    likelihood_list.append(likelihood_dict)

likelihood_list[0]

> /tmp/ipykernel_2657358/2882572435.py(8)<module>()
      6     logit_tensor = torch.stack([logit_dict[k] for k in "ACGT"], dim=1)
      7     import pdb; pdb.set_trace()
----> 8     likelihood_tensor = torch.softmax(logit_tensor, dim=1)
      9     likelihood_dict = {k: likelihood_tensor[i] for i, k in enumerate("ACGT")}
     10     likelihood_list.append(likelihood_dict)



torch.Size([6, 4])
torch.Size([6])
tensor([[ 0.3503, -0.0261,  0.4168, -0.2667],
        [ 0.1109,  0.3540,  0.4671,  0.0614],
        [ 0.5503, -0.3431,  0.3895, -0.2464],
        [-0.1649,  0.5716,  0.1255,  0.5956],
        [ 0.6182,  0.4910,  0.1161, -0.1491],
        [ 0.1694,  0.3965,  0.3556, -0.1738]])
tensor([[ 0.3503,  0.1109,  0.5503, -0.1649,  0.6182,  0.1694],
        [-0.0261,  0.3540, -0.3431,  0.5716,  0.4910,  0.3965],
        [ 0.4168,  0.4671,  0.3895,  0.1255,  0.1161,  0.3556],
        [-0.2667,  0.0614, -0.2464,  0.5956, -0.1491, -0.1738]])
tensor([[ 0.3503,  0.1109,  0.5503, -0.1649,  0.6182,  0.1694],
        [-0.0261,  0.3540, -0.3431,  0.5716,  0.4910,  0.3965],
        [ 0.4168,  0.4671,  0.3895,  0.1255,  0.1161,  0.3556],
        [-0.2667,  0.0614, -0.2464,  0.5956, -0.1491, -0.1738]])
tensor([[ 0.3503,  0.1109,  0.5503, -0.1649,  0.6182,  0.1694],
        [-0.0261,  0.3540, -0.3431,  0.5716,  0.4910,  0.3965],
        [ 0.4168,  0.4671,  0.3895,  0.1255,  

In [42]:
# Log likelihood ratios are used to score whether a substitution is more likely under one model than another
# For example, the log likelihood ratio of a C to T substitution at position 2 in the first sequence is:
logits[0]["T"][1] - logits[0]["C"][1]

tensor(-0.2926)